In [ ]:
!pip install langchain boto3
!pip install transformers
!pip install sentence_transformers

In [ ]:
pip install langchain==0.0.224

In [ ]:
pip install boto3 --upgrade

In [ ]:
import boto3
from langchain.retrievers import AmazonKendraRetriever
from typing import List
from typing import Dict
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain import SagemakerEndpoint, LLMChain
from langchain.llms.sagemaker_endpoint import LLMContentHandler
from langchain.chains.question_answering import load_qa_chain
import json

from langchain.docstore.document import Document
from langchain.memory import ConversationBufferWindowMemory
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

In [ ]:
class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, inputs: list[str], model_kwargs: Dict) -> bytes:
        input_str = json.dumps({"inputs": inputs, **model_kwargs})
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> List[List[float]]:
        response_json = json.loads(output.read().decode("utf-8"))
#        return response_json["vectors"]
        return response_json[0]["generated_text"]



content_handler = ContentHandler()


In [ ]:
from langchain import PromptTemplate, LLMChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter


llm2 = SagemakerEndpoint(
endpoint_name="hf-llm-falcon-40b-instruct-bf16-2023-06-23-19-34-33-102",
#endpoint_name="hf-llm-falcon-7b-bf16-2023-06-24-20-08-14-262",
#endpoint_name="hf-llm-falcon-40b-bf16-2023-06-24-20-20-44-608",
model_kwargs={
     "parameters" : {"do_sample": False,
    "top_p": 0.9,
    "temperature": 0.1,
    "max_new_tokens": 200
              }},
region_name="us-east-1",
content_handler=content_handler
)

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 4000,
    chunk_overlap  = 100,
    length_function = len,
)

In [ ]:
with open("bots.txt") as f:
    state_of_the_union = f.read()

docs = text_splitter.create_documents([state_of_the_union])
#print(texts)

In [ ]:
from langchain.docstore.document import Document

for x in docs:
    print(x)
    print("---")

In [ ]:
map_prompt_template = """Write a short sentence single line summary for following text:


{text}


Summary:"""


reduce_prompt_template = """Write a summary paragraph of the following:


{text}


Summary:"""

In [ ]:
from langchain.chains.summarize import load_summarize_chain

MAP_PROMPT = PromptTemplate(template=map_prompt_template, input_variables=["text"])
REDUCE_PROMPT = PromptTemplate(template=reduce_prompt_template, input_variables=["text"])



chain = load_summarize_chain(llm2, chain_type="map_reduce", return_intermediate_steps=True, map_prompt=MAP_PROMPT,  combine_prompt=REDUCE_PROMPT)
#chain.run(docs)
chain({"input_documents": docs}, return_only_outputs=True)